# 

In [1]:
# Step 1: Import necessary libraries
import os
import cv2
import numpy as np
import pickle
import joblib
import torch
import torch.nn.functional as F
from torchvision import transforms
from transformers import ViTModel, ViTFeatureExtractor
import pywt
# Step 2: Load Saved Models and Feature Files
# Face
face_model = joblib.load("random_forest_face_model.pkl")
with open("face_features_and_labels.pkl", 'rb') as f:
    face_data = pickle.load(f)
face_encoder = face_data["label_encoder"]
# Iris
iris_model = joblib.load("rf_iris_model_wtn_vit.pkl")
with open("iris_features_wtn_vit.pkl", 'rb') as f:
    iris_data = pickle.load(f)
iris_encoder = iris_data["label_encoder"]
# Fingerprint
finger_model = joblib.load("random_forest_fingerprint_model_cnn_wtn.pkl")
with open("fingerprint_features_cnn_wtn.pkl", 'rb') as f:
    finger_data = pickle.load(f)
finger_encoder = finger_data["label_encoder"]
# Step 3: Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Step 4: Load CNN model used for fingerprint feature extraction
import torch.nn as nn
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 256)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56)
        x = F.relu(self.fc1(x))
        return x
finger_cnn_model = SimpleCNN().to(device)
finger_cnn_model.eval()
# Step 5: ViT Setup
vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k').to(device).eval()
vit_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
# Step 6: Feature Extraction Functions
# --- Face ---
cnn_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
cnn_model = nn.Sequential(*list(cnn_model.children())[:-1]).to(device).eval()
cnn_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
def extract_face_features(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    image_rgb = cv2.cvtColor(cv2.resize(image, (224, 224)), cv2.COLOR_BGR2RGB)
    # CNN features
    img_tensor = cnn_transform(image_rgb).unsqueeze(0).to(device)
    with torch.no_grad():
        cnn_feat = cnn_model(img_tensor).squeeze().cpu().numpy()
    # ViT features
    inputs = vit_extractor(images=image_rgb, return_tensors="pt").to(device)
    with torch.no_grad():
        vit_feat = vit_model(**inputs).pooler_output.squeeze().cpu().numpy()
    combined = np.concatenate((cnn_feat, vit_feat))  # 2048 + 768
    return combined
# --- Iris ---
def wavelet_transform(image):
    coeffs = pywt.wavedec2(image, 'haar', level=2)
    flattened = []
    for coeff in coeffs:
        if isinstance(coeff, tuple):
            for arr in coeff:
                flattened.extend(arr.flatten())
        else:
            flattened.extend(coeff.flatten())
    return np.array(flattened[:4096])
def extract_iris_features(image_path):
    image = cv2.imread(image_path)
    if image is None:
        return None
    image_resized = cv2.resize(image, (224, 224))
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
    # WTN features
    wavelet_feat = wavelet_transform(gray)
    # ViT features
    inputs = vit_extractor(images=image_resized, return_tensors="pt").to(device)
    with torch.no_grad():
        vit_feat = vit_model(**inputs).pooler_output.squeeze().cpu().numpy()
    combined = np.concatenate((wavelet_feat, vit_feat))  # 4096 + 768
    return combined
# --- Fingerprint ---
transform_finger = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
def extract_fingerprint_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return None
    image_resized = cv2.resize(image, (224, 224))
    # WTN features
    wtn_feat = wavelet_transform(image_resized)[:2048]  # :white_check_mark: Force only 2048 features
    # CNN features
    input_tensor = transform_finger(image_resized).unsqueeze(0).to(device)
    with torch.no_grad():
        cnn_feat = finger_cnn_model(input_tensor).cpu().numpy().flatten()
    cnn_feat = cnn_feat[:256]  # :white_check_mark: Force only 256 features
    # Combine both features
    combined = np.concatenate((wtn_feat, cnn_feat))  # 2048 + 256 = 2304
    return combined
# Step 7: Predict Function
def predict_user(face_path, iris_path, fingerprint_path):
    print("\nPredicting from Face...")
    face_features = extract_face_features(face_path)
    face_pred = face_model.predict([face_features])[0]
    face_label = face_encoder.inverse_transform([face_pred])[0]
    print(f"Face Prediction: {face_label}")
    print("\nPredicting from Iris...")
    iris_features = extract_iris_features(iris_path)
    iris_pred = iris_model.predict([iris_features])[0]
    iris_label = iris_encoder.inverse_transform([iris_pred])[0]
    print(f"Iris Prediction: {iris_label}")
    print("\nPredicting from Fingerprint...")
    fingerprint_features = extract_fingerprint_features(fingerprint_path)
    finger_pred = finger_model.predict([fingerprint_features])[0]
    finger_label = finger_encoder.inverse_transform([finger_pred])[0]
    print(f"Fingerprint Prediction: {finger_label}")
    # Final Decision
    if face_label == iris_label == finger_label:
        print("\n:white_check_mark: Person is AUTHORIZED")
        return True
    else:
        print("\n:no_entry_sign: Person is NOT AUTHORIZED")
        return False
# Step 8: Run Prediction
if __name__ == "__main__":
    face_img = r"C:\Users\rames\OneDrive\Desktop\Biometrics\data\face\Test\Aparna\Aparna_1.jpg"
    iris_img = r"C:\Users\rames\OneDrive\Desktop\Biometrics\data\iris\Test\Aparna\Aparna_1.jpg"
    finger_img = r"C:\Users\rames\OneDrive\Desktop\Biometrics\data\fingerprint\Test\Aparna\Aparna_1.jpg"
    predict_user(face_img, iris_img, finger_img)


C:\Users\rames\anaconda3\envs\Biometrics\lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Using cache found in C:\Users\rames/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\rames\anaconda3\envs\Biometrics\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\rames\anaconda3\envs\Biometrics\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warni


Predicting from Face...
Face Prediction: Aparna

Predicting from Iris...
Iris Prediction: Aparna

Predicting from Fingerprint...
Fingerprint Prediction: Aparna

:white_check_mark: Person is AUTHORIZED
